<a href="https://colab.research.google.com/github/emrapport/w266-final-project/blob/master/modeling/CNN_w_contraction_dict_with_seed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Duplicate of CNN_w_contraction_dict but with set seed
11/3/2019
Overview: this was the first model I trained where I had dealt with the contraction stuff (by using dictionary to expand contractions, then removing additional apostrophes from training set). We see fewer unknown words know. Results seem relatively similar to previous iterations of model - validation accuracy bounces back and forth, hovering at around where our LR baseline was, maybe a little better. Here we see validation accuracy really dropping after a certain place - could also have to do with the fact that we're using a somewhat smaller training set. 

In [0]:
# this notebook is based off of this blog post: 
# https://realpython.com/python-keras-text-classification/#reader-comments

In [0]:
DATASET_NAME = "contractions"
MODEL_NAME = "contractions"
MAX_SEQ_LENGTH = 20
TRAINING_SET_SIZE = 2000000
VAL_SET_SIZE = 1000000
NUM_EPOCHS = 15
BATCH_SIZE = 1000

# I don't think we have a good reason to do this right now
# but we might eventually 
SAVE_TOKENIZED_DATA = False

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import HashingVectorizer
from scipy.sparse import hstack, vstack
from keras.preprocessing.text import Tokenizer
from tensorflow.keras import Sequential, layers
from keras.utils import plot_model
import pandas as pd
import numpy as np
import copy
import time
import pickle
!pip install gcsfs

pd.set_option('max_colwidth', 100)

Using TensorFlow backend.


     |████████████████████████████████| 71kB 4.4MB/s 
  Found existing installation: fsspec 0.5.2
    Uninstalling fsspec-0.5.2:
      Successfully uninstalled fsspec-0.5.2


In [3]:
# this cell is only necessary if running in colab
project_id = 'w266-251323'
import uuid
bucket_name = 'fb-congressional-data/'
from google.colab import auth
auth.authenticate_user()
!gcloud config set project {project_id}

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Updated property [core/project].


In [4]:
train_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/train.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
dev_df = pd.read_csv("gs://fb-congressional-data/contraction_expanded_data/dev.csv", index_col=0)

/usr/local/lib/python3.6/dist-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [0]:
# Set seeds for reproducible results.
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [0]:
# shuffle the data
# be sure to do this before you extract X's and y's!!
train_df = train_df.sample(frac=1)
dev_df = dev_df.sample(frac=1)

In [8]:
train_df.op_gender.value_counts()

M    7024476
W    2854540
Name: op_gender, dtype: int64

In [9]:
dev_df.op_gender.value_counts()

M    1948371
W     344536
Name: op_gender, dtype: int64

In [0]:
y_train = train_df.op_gender.values
y_dev = dev_df.op_gender.values

In [0]:
def turn_to_ints(li):
    final_list = []
    for gender in li:
        if gender=='M':
            final_list.append(1)
        else:
            final_list.append(0)
    return final_list
            
y_train = turn_to_ints(y_train)
y_dev = turn_to_ints(y_dev)

In [0]:
y_train = np.asarray(y_train)
y_dev = np.asarray(y_dev)

In [0]:
def get_text_list(init_list):
    sentences = []
    for sentence in init_list:
        if type(sentence) != str:
            sentences.append("")
        else:
            sentences.append(sentence)
    return sentences

new_sentences_train = get_text_list(train_df.response_text.values)
new_sentences_test = get_text_list(dev_df.response_text.values)

In [14]:
time_start = time.time()

# this is the default list of filters + apostrophe
# added because we have dealt with common contractions, so other apostrophes should mostly be possessive 
tokenizer = Tokenizer(num_words=200000, filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(new_sentences_train)

X_train = tokenizer.texts_to_sequences(new_sentences_train)
X_test = tokenizer.texts_to_sequences(new_sentences_test)

vocab_size = len(tokenizer.word_index) + 1

currentTime = time.gmtime(time.time() - time_start)

#Convert the gmtime struct to a string
timeStr = time.strftime("%M minutes, %S seconds", currentTime)

print("Tokenized in {}".format(timeStr))

Tokenized in 11 minutes, 00 seconds


In [0]:
from keras.preprocessing.sequence import pad_sequences

maxlen = 20

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [0]:
if SAVE_TOKENIZED_DATA:
  x_train_path = 'X_train_{}.pkl'.format(DATASET_NAME)
  x_dev_path = 'X_dev_{}.pkl'.format(DATASET_NAME)
  y_train_path = 'y_train_{}.pkl'.format(DATASET_NAME)
  y_dev_path = 'y_dev_{}.pkl'.format(DATASET_NAME)

  with open(x_train_path, 'wb') as file:
      pickle.dump(X_train, file)   
  with open(x_dev_path, 'wb') as file:
      pickle.dump(X_test, file)
  with open(y_train_path, 'wb') as file:
      pickle.dump(y_train, file)
  with open(y_dev_path, 'wb') as file:
      pickle.dump(y_dev, file)

  # copy to bucket
  !gsutil cp /content/{x_train_path} gs://fb-congressional-data/test
  !gsutil cp /content/{x_dev_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_train_path} gs://fb-congressional-data/
  !gsutil cp /content/{y_dev_path} gs://fb-congressional-data/

In [17]:
!gsutil cp gs://fb-congressional-data/glove* /tmp/

Copying gs://fb-congressional-data/glove.6B.100d.txt...
Copying gs://fb-congressional-data/glove.6B.200d.txt...
Copying gs://fb-congressional-data/glove.6B.300d.txt...
Copying gs://fb-congressional-data/glove.6B.50d.txt...
- [4 files][  2.1 GiB/  2.1 GiB]   56.2 MiB/s                                   
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://fb-congressional-data/glove.6B.zip...
| [5 files][  2.9 GiB/  2.9 GiB]   22.5 MiB/s                                   
Operation completed over 5 objects/2.9 GiB.                                      


In [0]:
def create_embedding_matrix(filepath, word_index, embedding_dim):
    vocab_size = len(word_index) + 1  # Adding again 1 because of reserved 0 index
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    with open(filepath) as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word] 
                embedding_matrix[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]

    return embedding_matrix

embedding_dim = 100
embedding_matrix = create_embedding_matrix(
                     '/tmp/glove.6B.{}d.txt'.format(embedding_dim),
                      tokenizer.word_index, embedding_dim)

In [19]:
# trying to figure out which words are empty here
counter = 0
empty_indexes = []
for index, row in enumerate(embedding_matrix):
  if sum(row) == 0:
    empty_indexes.append(index)
    counter += 1
  if counter > 1000:
    break

for idx in empty_indexes:
  try:
    print(tokenizer.index_word[idx])
  except:
    print("No entry for {}".format(idx))


No entry for 0
stillsanders
youtu
fbid
palestinei
•
standwithrand
🇺🇸
obummer
rinos
🇺🇸🇺🇸
doyourjob
huffingtonpost
👍
washingtonpost
feelthebern
😍
lmao
killary
❤️
fbcdn
nobillnobreak
hphotos
bridenstine
randrally
hahaha
isil
randpaul
cispa
bengazi
😊
sheeple
bernieorbust
😂
neverhillary
repub
aspx
😡
thehill
1073741828
dailycaller
trump2016
libtards
obozo
thinkprogress
§
scontent
͡°
❤
politicususa
berniesanders
agsaf
westernjournalism
╬═╬
sphotos
drumpf
2366
👎
washingtontimes
worstresponders
teabaggers
hahahaha
fref
akamaihd
wenstrup
hilliary
demandavote
libtard
nobama
oboma
banksters
teaparty
koolaid
ovomit
nationalreview
infowars
notif
😍😍😍
shouldnt
🇺🇸🇺🇸🇺🇸
bernie2016
protectsacredlands
opensecrets
occupydemocrats
0bama
😍😍
daca
2fwww
devoss
nodapl
imho
♥
washingtonexaminer
dumbass
wtg
dapl
███
govtrack
obuma
killery
cromnibus
cnsnews
على
conservativetribune
😉
gofundme
lieing
😀
amodei
bucshon
🙏
benifits
democraps
bonamici
democrates
♡
p71bjjcp8
randpaul2016
felicidades
abcnews
tahmooressi
264

In [20]:
# hmmmm....
nonzero_elements = np.count_nonzero(np.count_nonzero(embedding_matrix, axis=1))
nonzero_elements / vocab_size

0.16257338124855325

In [21]:
embedding_matrix.shape

(1054084, 100)

In [22]:
model = Sequential()
model.add(layers.Embedding(vocab_size, embedding_dim, 
                           weights=[embedding_matrix], 
                           input_length=maxlen, 
                           trainable=False))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           105408400 
_________________________________________________________________
conv1d (Conv1D)              (None, 16, 128)           64128     
_________________________________________________________________
global_max_pooling1d (Global (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
_________________________________________________________________
dense_1 (Dense)            

In [0]:
smaller_X_train = X_train[:TRAINING_SET_SIZE]

In [0]:
smaller_y_train = y_train[:TRAINING_SET_SIZE]

In [0]:
smaller_X_dev = X_test[:VAL_SET_SIZE]
smaller_y_dev = y_dev[:VAL_SET_SIZE]

In [26]:
try:
  time_start = time.time()

  history = model.fit(smaller_X_train, smaller_y_train,
                      epochs=NUM_EPOCHS,
                      verbose=True,
                      validation_data=(smaller_X_dev, smaller_y_dev),
                      batch_size=BATCH_SIZE)

  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))

except:
  currentTime = time.gmtime(time.time() - time_start)

  #Convert the gmtime struct to a string
  timeStr = time.strftime("%M minutes, %S seconds", currentTime)

  print("Trained in {}".format(timeStr))  

Train on 2000000 samples, validate on 1000000 samples
Epoch 1/15
2000000/2000000 [==============================] - 62s 31us/sample - loss: 0.5299 - acc: 0.7486 - val_loss: 0.4527 - val_acc: 0.8327
Epoch 2/15
2000000/2000000 [==============================] - 60s 30us/sample - loss: 0.5076 - acc: 0.7591 - val_loss: 0.4889 - val_acc: 0.8114
Epoch 3/15
2000000/2000000 [==============================] - 60s 30us/sample - loss: 0.5006 - acc: 0.7623 - val_loss: 0.4368 - val_acc: 0.8410
Epoch 4/15
2000000/2000000 [==============================] - 62s 31us/sample - loss: 0.4960 - acc: 0.7646 - val_loss: 0.4634 - val_acc: 0.8267
Epoch 5/15
2000000/2000000 [==============================] - 61s 30us/sample - loss: 0.4927 - acc: 0.7661 - val_loss: 0.4424 - val_acc: 0.8375
Epoch 6/15
2000000/2000000 [==============================] - 62s 31us/sample - loss: 0.4897 - acc: 0.7675 - val_loss: 0.4816 - val_acc: 0.8117
Epoch 7/15
2000000/2000000 [==============================] - 60s 30us/sample - lo

In [27]:
loss, accuracy = model.evaluate(smaller_X_train, smaller_y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_dev, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))


Training Accuracy: 0.7759
Testing Accuracy:  0.8211


In [0]:
preds = model.predict(X_test)

In [0]:
dev_df['probs'] = preds

In [0]:
def pred_to_label(row):
  if row['probs'] >= .5:
    return 'M'
  else:
    return 'W'

dev_df['preds'] = dev_df.apply(pred_to_label, axis=1)

In [0]:
wrong_preds = dev_df[dev_df.op_gender!=dev_df.preds]

In [32]:
wrong_preds.op_gender.value_counts()

W    301769
M    108469
Name: op_gender, dtype: int64

In [33]:
# total proportion of preds for women
dev_df.preds.value_counts()['W'] / len(dev_df)

0.06595819193713483

In [34]:
#random sample of mistakes
wrong_preds.head(20)

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1927625,10977017,86569077,W,389345,Jane,you sounded like a fool.,Kirsten Gillibrand,Congress_Democratic,0.813345,M
2078802,11219782,77234050,W,398978,Kathy,"was not finished with my post, i want the military to recognized that fort mccellan alabama is a...",Cathy McMorris Rodgers,Congress_Republican,0.771251,M
375560,3125651,23984618,W,82876,Carolyn,i feel better knowing you will get nothing done in this committee based on your history of doing...,Ann Wagner,Congress_Republican,0.660079,M
913217,5162271,46506154,M,161127,Elizabeth,happy birthday and wishes for blessings for you and yours!,Marco Rubio,Congress_Republican,0.449775,W
2153523,11294503,77234050,W,400424,Barb,so tell me again why we should bail them out!,Cathy McMorris Rodgers,Congress_Republican,0.886648,M
930243,5179297,46506154,M,161214,Marco,"merry christmas, senator! to you and your beautiful family.",Marco Rubio,Congress_Republican,0.373254,W
1940138,10989530,86569077,W,389446,Joanne,if you want someone to lie to your face...who thinks it has okay to murder a baby in the womb ge...,Kirsten Gillibrand,Congress_Democratic,0.795523,M
1927800,10977192,86569077,W,389346,Jeanne,thank you for representing me!,Kirsten Gillibrand,Congress_Democratic,0.617999,M
2126066,11267046,77234050,W,399739,Hank,this letter certainly makes it clear that the republican house-caused government shutdown is all...,Cathy McMorris Rodgers,Congress_Republican,0.801024,M
1557113,7975407,99469231,M,296684,Dalia,we love you!,John McCain,Congress_Republican,0.235096,W


In [35]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=False)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
2136861,11277841,77234050,W,399896,Carol,"please senator mcmorris, support rand paul in his efforts to protect our rights to due process",Cathy McMorris Rodgers,Congress_Republican,0.999972,M
2007127,11056519,86569077,W,390101,Samantha,"thank you senator gillibrand, and also thanks to senator schumer.",Kirsten Gillibrand,Congress_Democratic,0.999920,M
1993458,11042850,86569077,W,389980,Kristin,"that is why i stand by rand. but you must be constitutional across the bridge and back, not sel...",Kirsten Gillibrand,Congress_Democratic,0.999896,M
2050249,11099641,86569077,W,390682,Tina,thank god for senator chuck schumer!,Kirsten Gillibrand,Congress_Democratic,0.999849,M
1909593,10958985,86569077,W,389336,Danielle,thank you kirsten gillibrand for fighting for and representing new york!,Kirsten Gillibrand,Congress_Democratic,0.999820,M
1963043,11012435,86569077,W,389663,Charlene,"you hit the nail on the head....its the ""american dream"" right? its sad to see how greedy our le...",Kirsten Gillibrand,Congress_Democratic,0.999819,M
2003832,11053224,86569077,W,390071,Susan,"yes, thank goodness for rand paul! wasting the tax payers money!",Kirsten Gillibrand,Congress_Democratic,0.999814,M
2065901,11205655,48870736,W,395025,Kriss,thank you representative mccollum. i support this issue 110%.,Betty McCollum,Congress_Democratic,0.999810,M
2020045,11069437,86569077,W,390207,Greto,thank you senator schumer,Kirsten Gillibrand,Congress_Democratic,0.999807,M
2160151,11301131,77234050,W,400749,Larry,great job justin. what an ispiration to others out there. thank you for supporting our greates...,Cathy McMorris Rodgers,Congress_Republican,0.999770,M


In [36]:
# most confident wrong predictions where the answer was F but they thought M
# same as before - references to a male politician 
wrong_preds.sort_values('probs', ascending=True)[:20]

,original_idx,op_id,op_gender,post_id,responder_id,response_text,op_name,op_category,probs,preds
1373869,7792163,99469231,M,296202,Michael,as the terrorist attacks abroad continue....here at home (ignorance) on the behalf of state leve...,John McCain,Congress_Republican,0.000016,W
181521,2358273,62246450,M,52502,Annie,keep going elizabeth! we support you!,Ron Wyden,Congress_Democratic,0.000019,W
944650,5193704,46506154,M,161264,Suzette,i trust my congresswoman marsha blackburn to do the right thing on this.,Marco Rubio,Congress_Republican,0.000063,W
264413,2528948,59227815,M,63410,Rhonda,"love you, mia love! almost-aborted first black congresswoman.",Paul Ryan,Congress_Republican,0.000098,W
1857510,10890438,10458808,M,385771,Kat,"thank you, congresswoman pelosi! 💖",Mark Takano,Congress_Democratic,0.000115,W
1883166,10916094,10458808,M,385912,Stephen,but takano does not support kamala harris! he just lost my vote!,Mark Takano,Congress_Democratic,0.000122,W
210220,2386972,62246450,M,52711,John,"run, liz, run!",Ron Wyden,Congress_Democratic,0.000168,W
2204532,11908709,17310,M,440492,Debbie,a great congresswoman!,Sander Levin,Congress_Democratic,0.000177,W
1870185,10903113,10458808,M,385776,Hannah,i see tulsi!,Mark Takano,Congress_Democratic,0.000195,W
685807,4934861,46506154,M,160245,Michael,trump vs kanye 2020,Marco Rubio,Congress_Republican,0.000209,W
